In [1]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.models import Model
import numpy as np 
import pandas as pd 
import re
import nltk
from preprocess import *
from models import *

2022-12-02 14:17:50.032968: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 14:17:51.562438: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2022-12-02 14:17:51.562735: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2022-12-02 14:17:51.562746: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] 

In [2]:
df = pd.read_csv("questions.csv")
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
df.shape

(404351, 6)

In [4]:
#pick a sample of 5000 distinct random rows
df = df.sample(n=100000, random_state=1)
df.shape

(100000, 6)

In [5]:
# question_1, question_2 = df['question1'].to_list(), df['question2'].to_list()
# is_duplicate = df['is_duplicate'].to_list()
# preprocess_neural(question_1, question_2, is_duplicate)

In [6]:
q1_inp, q2_inp, is_duplicate = df['question1'].to_list(), df['question2'].to_list(), df['is_duplicate'].to_list()

Acquired Test data

In [7]:
MAX_WORDS_VOCAB = 200000
tokenizer = Tokenizer(num_words = MAX_WORDS_VOCAB, lower=False, split=" ")
tokenizer.fit_on_texts(list(df['question1'].values.astype(str))+list(df['question2'].values.astype(str)))

In [8]:
print("Number of words in vocabulary: ", len(tokenizer.word_index))

Number of words in vocabulary:  60949


In [9]:

q1_sequence = tokenizer.texts_to_sequences(df['question1'].values.astype(str))
q1_sequence = pad_sequences(q1_sequence, maxlen = 128)

q2_sequence = tokenizer.texts_to_sequences(df['question2'].values.astype(str))
q2_sequence = pad_sequences(q2_sequence, maxlen = 128)

In [10]:
windex = tokenizer.word_index

In [11]:
embedding_index = {}
with open('glove.6B.300d.txt','r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], 'float32')
        embedding_index[word] = vectors
    f.close()

In [12]:
embedding_matrix = np.random.random((len(windex)+1, 300))

for word, i in windex.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)

(60950, 300)


In [13]:
print(len(windex)+1)

60950


In [14]:
#split the data into 70-20-10 train-validation-test with random state 42
from sklearn.model_selection import train_test_split
q1_train, q1_test, q2_train, q2_test, y_train, y_test = train_test_split(q1_sequence, q2_sequence, is_duplicate, test_size=0.1, random_state=42)
q1_train, q1_val, q2_train, q2_val, y_train, y_val = train_test_split(q1_train, q2_train, y_train, test_size=0.2, random_state=42)

In [15]:
import tensorflow as tf
y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=2)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=2)

In [16]:
is_duplicate = np.array(is_duplicate)
print(is_duplicate.shape)

(100000,)


In [17]:
#print the ratio of positive and negative samples in train, validation and test
y_train, y_val, y_test = np.array(y_train), np.array(y_val), np.array(y_test)
print("Train: ", sum(y_train)/len(y_train))
print("Validation: ", sum(y_val)/len(y_val))
print("Test: ", sum(y_test)/len(y_test))

Train:  [0.63294444 0.36705556]
Validation:  [0.6358333  0.36416668]
Test:  [0.6291 0.3709]


In [19]:
model = LsTM(emb_mat = embedding_matrix, vocab_size = len(windex)+1)

2022-12-02 14:18:17.626672: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-02 14:18:17.626735: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2022-12-02 14:18:17.627957: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
model.train_model()

In [21]:
model = LsTM(embedding_matrix, len(windex) + 1, loss="categorical_crossentropy")

In [22]:
model.train_model()

In [23]:
model.get_model_summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 128, 300)     18285000    ['input_3[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, 128, 300)     18285000    ['input_4[0][0]']                
                                                                                            

In [24]:
print(q1_val.shape)

(18000, 128)


In [25]:
model.model.fit([q1_train, q2_train], y_train, epochs = 4, validation_data = ([q1_val, q2_val], y_val), batch_size = 8, validation_batch_size=4, verbose = 1)

Epoch 1/4
9000/9000 [==============================] - 1342s 149ms/step - loss: 0.5511 - accuracy: 0.7175 - val_loss: 0.5226 - val_accuracy: 0.7364
Epoch 2/4
9000/9000 [==============================] - 1363s 151ms/step - loss: 0.4874 - accuracy: 0.7626 - val_loss: 0.5049 - val_accuracy: 0.7547
Epoch 3/4
9000/9000 [==============================] - 1333s 148ms/step - loss: 0.4301 - accuracy: 0.7970 - val_loss: 0.4883 - val_accuracy: 0.7714
Epoch 4/4
9000/9000 [==============================] - 1315s 146ms/step - loss: 0.3721 - accuracy: 0.8288 - val_loss: 0.4709 - val_accuracy: 0.7901


In [26]:
y_pred = model.predict([q1_test, q2_test])
y_pred1d, y_actual1d = [], []
for i in range(len(y_test)):
    if(y_test[i][0] == 1):
        y_actual1d.append(0)
    else:
        y_actual1d.append(1)

for i in range(len(y_pred)):
    if(y_pred[i][0] > y_pred[i][1]):
        y_pred1d.append(0)
    else:
        y_pred1d.append(1)

from sklearn.metrics import accuracy_score, f1_score
print("Accuracy: ", accuracy_score(y_actual1d, y_pred1d))
print("F1 Score: ", f1_score(y_actual1d, y_pred1d))

313/313 [==============================] - 25s 71ms/step
Accuracy:  0.784100015
F1 Score:  0.7357100716392751


: 